In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from transformers import AutoFeatureExtractor, SwinForImageClassification

# Check if GPU is available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define transformations for the training and testing data
train_transform = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

test_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load the datasets with ImageFolder
train_dataset = datasets.ImageFolder("D:/machine  learning/face dataset/train_set", transform=train_transform)
test_dataset = datasets.ImageFolder("D:/machine  learning/face dataset/test_set", transform=test_transform)

# Define the dataloaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Load the Swin Transformer model and feature extractor from Hugging Face
model_name = "microsoft/swin-tiny-patch4-window7-224"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_name)
model = SwinForImageClassification.from_pretrained(model_name)

# Modify the model for binary classification
model.classifier = nn.Linear(model.config.hidden_size, 1)  # Replace classifier with binary output layer

# Move model to appropriate device
model.to(device)

# Define loss function and optimizer
criterion = nn.BCEWithLogitsLoss()  # Use BCEWithLogitsLoss for binary classification
optimizer = optim.Adam(model.parameters(), lr=0.001)

print("Using device:", device)

# Training the model
epochs = 25
for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device).float().view(-1, 1)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(images).logits
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f'Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(train_loader):.4f}')

# Testing the model
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device).float().view(-1, 1)  # Move data to GPU
        outputs = model(images).logits
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
print(f'Accuracy: {100 * correct / total:.2f}%')

# Single image prediction
from PIL import Image
single_image = Image.open("C:/Users/rahul/Downloads/George-W-Bush.jpeg")
single_image = test_transform(single_image).unsqueeze(0).to(device)  # Move data to GPU
model.eval()
with torch.no_grad():
    result = model(single_image).logits
    if torch.sigmoid(result).item() > 0.5:
        prediction = 'authorized'
    else:
        prediction = 'unauthorized'
print(prediction)
